In [1]:
%matplotlib inline
import matplotlib.pylab as plt

In [2]:
import numpy as np

In [3]:
from pycrates import read_file

In [4]:
from coords.utils import point_separation

In [5]:
tab = read_file("obspar.fits")

In [6]:
obsid = tab.obs_id.values
obi   = tab.obi_num.values
inst  = tab.instrume.values
det   = tab.detnam.values
grt   = tab.grating.values
obsmode = tab.obs_mode.values
dmode = tab.datamode.values
rmode = tab.readmode.values
ra = tab.ra_pnt.values
dec = tab.dec_pnt.values

In [7]:
print(set(rmode[inst=='ACIS']))
print(set(dmode[inst=='ACIS']))
print(set(obsmode))

{'CONTINUOUS', 'TIMED'}
{'VFAINT', 'CC33_GRADED', 'FAINT_BIAS', 'CC33_FAINT', 'GRADED', 'FAINT', 'RAW'}
{'SECONDARY', 'POINTING'}


In [9]:
acis_only = np.where(inst=="ACIS")
te_only = np.where(rmode=="TIMED")
pointing = np.where(obsmode=="POINTING")

In [10]:
myobs, = np.where( (inst=='ACIS') & (rmode=='TIMED') & (obsmode=='POINTING') & (grt=='NONE'))

In [11]:
from collections import namedtuple
Obs = namedtuple( 'Obs', ['obsid','obinum', 'ra', 'dec'])

In [12]:
mylist = [Obs(obsid[i],obi[i],ra[i],dec[i]) for i in myobs]

In [13]:
SEPARATION_THRESHOLD = 20/3600.0 # arcsec diff in pointing

In [14]:
retval = {}
n_obs = len(mylist)
for ii in range(n_obs-1):    

    if (ii%500)==0:
        print(ii)

    obsA = mylist[ii]
    
    ra_a = obsA.ra
    dec_a = obsA.dec
    
    for jj in range(ii+1,n_obs):
        obsB = mylist[jj]
        ra_b = obsB.ra
        dec_b = obsB.dec

        if np.abs(dec_a-dec_b) > 3*SEPARATION_THRESHOLD:
            continue        
        
        d = point_separation(ra_a, dec_a, ra_b,dec_b)
        if d <= SEPARATION_THRESHOLD:
            if obsA in retval:
                retval[obsA].append(obsB)
            else:
                retval[obsA]=[obsB]
            
            if obsB in retval:
                retval[obsB].append(obsA)
            else:
                retval[obsB]=[obsA]
            

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500


In [15]:
len(retval)

4706

In [16]:
len(myobs)

13589

In [17]:
ll = [len(retval[x]) for x in retval]

In [18]:
def myid(oo):
    rr = "o{}_{}".format(oo.obsid,oo.obinum)
    return(rr)
   

In [19]:
with open("match_no_tg.dat","w") as fp:
    fp.write( "digraph chandra { \n" )
    
    
    for k in retval:
        mm = myid(k)
        ll = ",".join([myid(x) for x in retval[k]])
        fp.write("{} -> {{ {} }};\n".format(mm,ll))
    
    fp.write("}\n")

In [20]:
%%bash 
neato  -Tpng match_no_tg.dat  > foo_no_tg.png

In [101]:
#from IPython.display import Image
#Image(filename='foo.png') 


In [21]:
!eog foo_no_tg.png
